In [5]:
# ===== Imports =====
import pandas as pd
import numpy as np

In [6]:
# ===== Task 1 - Create DataFrames & save to CSV =====

# Project dataframe - note: some rows have missing cost values (use np.nan)
project_rows = [
    ('A001','Project 1',1002000,'Finished'),
    ('A002','Project 2',2000000,'Ongoing'),
    ('A003','Project 3',4500000,'Finished'),
    ('A004','Project 4',5500000,'Ongoing'),
    ('A005','Project 5',np.nan,'Finished'),
    ('A002','Project 6',680000,'Failed'),
    ('A005','Project 7',400000,'Finished'),
    ('A003','Project 8',350000,'Failed'),
    ('A001','Project 9',np.nan,'Ongoing'),
    ('A003','Project 10',300000,'Finished'),
    ('A001','Project 11',2000000,'Failed'),
    ('A004','Project 12',1000000,'Ongoing'),
    ('A004','Project 13',3000000,'Finished'),
    ('A005','Project 14',200000,'Finished')
]
project = pd.DataFrame(project_rows, columns=['ID','Project','Cost','Status'])

print(project)
project.to_csv('Project_DF.csv', index=False)

# Employee dataframe data
employee_data = {
    'ID': ['A001','A002','A003','A004','A005'],
    'name': ['John Alter','Alice Luxumberg','Tom Sabestine','Nina Adgra','Amy Johny'],
    'Gender': ['M','F','M','F','F'],
    'City': ['Paris','London','Berlin','Newyork','Madrid'],
    'Age': [25,27,29,31,30]
}

employee = pd.DataFrame(employee_data)

print(employee)
employee.to_csv('Employee_DF.csv', index=False)

# Seniority Level dataframe
seniority_data = {
    'ID': ['A001','A002','A003','A004','A005'],
    'Designation Level': [2,2,3,2,3]
}
seniority = pd.DataFrame(seniority_data)

print(seniority)
seniority.to_csv('Seniority_DF.csv', index=False)

      ID     Project       Cost    Status
0   A001   Project 1  1002000.0  Finished
1   A002   Project 2  2000000.0   Ongoing
2   A003   Project 3  4500000.0  Finished
3   A004   Project 4  5500000.0   Ongoing
4   A005   Project 5        NaN  Finished
5   A002   Project 6   680000.0    Failed
6   A005   Project 7   400000.0  Finished
7   A003   Project 8   350000.0    Failed
8   A001   Project 9        NaN   Ongoing
9   A003  Project 10   300000.0  Finished
10  A001  Project 11  2000000.0    Failed
11  A004  Project 12  1000000.0   Ongoing
12  A004  Project 13  3000000.0  Finished
13  A005  Project 14   200000.0  Finished
     ID             name Gender     City  Age
0  A001       John Alter      M    Paris   25
1  A002  Alice Luxumberg      F   London   27
2  A003    Tom Sabestine      M   Berlin   29
3  A004       Nina Adgra      F  Newyork   31
4  A005        Amy Johny      F   Madrid   30
     ID  Designation Level
0  A001                  2
1  A002                  2
2  A003      

In [7]:
# ===== Task 2 - Replace missing project costs using running average (use for loop) =====

project = pd.read_csv('Project_DF.csv')

# Convert Cost to numeric and mark missing as NaN
project['Cost'] = pd.to_numeric(project['Cost'], errors='coerce')

# We'll fill NaN using running average of known neighbors. Requirement: use a for loop.
# Approach: For each row i with NaN, find previous non-NaN cost (search backward) and next non-NaN cost (search forward).

costs = project['Cost'].values.copy()

for i in range(len(costs)):
    if np.isnan(costs[i]):
        # find previous non-NaN
        prev_val = None
        for j in range(i-1, -1, -1):
            if not np.isnan(costs[j]):
                prev_val = costs[j]
                break
        # find next non-NaN
        next_val = None
        for j in range(i+1, len(costs)):
            if not np.isnan(costs[j]):
                next_val = costs[j]
                break
        # compute replacement
        if (prev_val is not None) and (next_val is not None):
            replacement = (prev_val + next_val) / 2.0
        elif prev_val is not None:
            replacement = prev_val
        elif next_val is not None:
            replacement = next_val
        else:
            replacement = np.nanmean(costs)
        costs[i] = replacement

project['Cost'] = costs
print(project)
project.to_csv('project_DF_filled.csv', index=False)

      ID     Project       Cost    Status
0   A001   Project 1  1002000.0  Finished
1   A002   Project 2  2000000.0   Ongoing
2   A003   Project 3  4500000.0  Finished
3   A004   Project 4  5500000.0   Ongoing
4   A005   Project 5  3090000.0  Finished
5   A002   Project 6   680000.0    Failed
6   A005   Project 7   400000.0  Finished
7   A003   Project 8   350000.0    Failed
8   A001   Project 9   325000.0   Ongoing
9   A003  Project 10   300000.0  Finished
10  A001  Project 11  2000000.0    Failed
11  A004  Project 12  1000000.0   Ongoing
12  A004  Project 13  3000000.0  Finished
13  A005  Project 14   200000.0  Finished


In [8]:
# ===== Task 3 - Split name into First Name & Last Name =====
employee = pd.read_csv('employee_DF.csv')

first_names = []
last_names = []
for full in employee['name']:
    parts = str(full).split()
    if len(parts) == 1:
        first_names.append(parts[0])
        last_names.append('')
    else:
        first_names.append(parts[0])
        last_names.append(' '.join(parts[1:]))

employee['First Name'] = first_names
employee['Last Name'] = last_names
employee = employee.drop(columns=['name'])

print(employee)
employee.to_csv('employee_DF_split.csv', index=False)

     ID Gender     City  Age First Name  Last Name
0  A001      M    Paris   25       John      Alter
1  A002      F   London   27      Alice  Luxumberg
2  A003      M   Berlin   29        Tom  Sabestine
3  A004      F  Newyork   31       Nina      Adgra
4  A005      F   Madrid   30        Amy      Johny


In [9]:
# ===== Task 4 - Join all three DataFrames into Final =====
# Read the saved files, then merge.

employee = pd.read_csv('employee_DF_split.csv')
seniority = pd.read_csv('Seniority_DF.csv')
project = pd.read_csv('project_DF_filled.csv')

# Merge employee with seniority on ID
final = pd.merge(employee, seniority, on='ID', how='left')
# Merge final with project (many-to-one since employee can have many projects)
final = pd.merge(final, project, on='ID', how='left')

cols_order = ['ID','First Name','Last Name','Gender','City','Age','Designation Level','Project','Cost','Status']
final = final[cols_order]
final.to_csv('Final.csv', index=False)
print(final)


      ID First Name  Last Name Gender     City  Age  Designation Level  \
0   A001       John      Alter      M    Paris   25                  2   
1   A001       John      Alter      M    Paris   25                  2   
2   A001       John      Alter      M    Paris   25                  2   
3   A002      Alice  Luxumberg      F   London   27                  2   
4   A002      Alice  Luxumberg      F   London   27                  2   
5   A003        Tom  Sabestine      M   Berlin   29                  3   
6   A003        Tom  Sabestine      M   Berlin   29                  3   
7   A003        Tom  Sabestine      M   Berlin   29                  3   
8   A004       Nina      Adgra      F  Newyork   31                  2   
9   A004       Nina      Adgra      F  Newyork   31                  2   
10  A004       Nina      Adgra      F  Newyork   31                  2   
11  A005        Amy      Johny      F   Madrid   30                  3   
12  A005        Amy      Johny      F 

In [10]:
# ===== Task 5 - Add Bonus column (5% for Finished projects) =====

final['Bonus'] = final['Cost'].apply(lambda x: round(0.05 * x,2) if not np.isnan(x) else 0)
# But only for Finished projects; set to 0 otherwise
final.loc[final['Status'] != 'Finished', 'Bonus'] = 0
final.to_csv('Final_with_bonus.csv', index=False)

print(final)

      ID First Name  Last Name Gender     City  Age  Designation Level  \
0   A001       John      Alter      M    Paris   25                  2   
1   A001       John      Alter      M    Paris   25                  2   
2   A001       John      Alter      M    Paris   25                  2   
3   A002      Alice  Luxumberg      F   London   27                  2   
4   A002      Alice  Luxumberg      F   London   27                  2   
5   A003        Tom  Sabestine      M   Berlin   29                  3   
6   A003        Tom  Sabestine      M   Berlin   29                  3   
7   A003        Tom  Sabestine      M   Berlin   29                  3   
8   A004       Nina      Adgra      F  Newyork   31                  2   
9   A004       Nina      Adgra      F  Newyork   31                  2   
10  A004       Nina      Adgra      F  Newyork   31                  2   
11  A005        Amy      Johny      F   Madrid   30                  3   
12  A005        Amy      Johny      F 

In [11]:
# ===== Task 6 - Demote designation for Failed projects and delete records with designation > 4 =====

final['Designation Level'] = final['Designation Level'].astype(float)  # ensure numeric
mask_failed = final['Status'] == 'Failed'
final.loc[mask_failed, 'Designation Level'] = final.loc[mask_failed, 'Designation Level'] + 1

# Drop employees whose designation level is above 4 (strictly > 4)
design_map = final.groupby('ID')['Designation Level'].first()
invalid_ids = design_map[design_map > 4].index.tolist()

final = final[~final['ID'].isin(invalid_ids)].copy()
final.to_csv('Final_after_demote.csv', index=False)
print(final)

      ID First Name  Last Name Gender     City  Age  Designation Level  \
0   A001       John      Alter      M    Paris   25                2.0   
1   A001       John      Alter      M    Paris   25                2.0   
2   A001       John      Alter      M    Paris   25                3.0   
3   A002      Alice  Luxumberg      F   London   27                2.0   
4   A002      Alice  Luxumberg      F   London   27                3.0   
5   A003        Tom  Sabestine      M   Berlin   29                3.0   
6   A003        Tom  Sabestine      M   Berlin   29                4.0   
7   A003        Tom  Sabestine      M   Berlin   29                3.0   
8   A004       Nina      Adgra      F  Newyork   31                2.0   
9   A004       Nina      Adgra      F  Newyork   31                2.0   
10  A004       Nina      Adgra      F  Newyork   31                2.0   
11  A005        Amy      Johny      F   Madrid   30                3.0   
12  A005        Amy      Johny      F 

In [12]:
# ===== Task 7 - Add Mr./Mrs. to First Name and drop Gender column =====

def add_title(row):
    fname = str(row['First Name'])
    if row['Gender'] == 'M':
        return 'Mr. ' + fname
    elif row['Gender'] == 'F':
        return 'Mrs. ' + fname
    else:
        return fname

final['First Name'] = final.apply(add_title, axis=1)
# Now drop Gender
final = final.drop(columns=['Gender'])
final.to_csv('Final_after_titles.csv', index=False)
print(final)

      ID  First Name  Last Name     City  Age  Designation Level     Project  \
0   A001    Mr. John      Alter    Paris   25                2.0   Project 1   
1   A001    Mr. John      Alter    Paris   25                2.0   Project 9   
2   A001    Mr. John      Alter    Paris   25                3.0  Project 11   
3   A002  Mrs. Alice  Luxumberg   London   27                2.0   Project 2   
4   A002  Mrs. Alice  Luxumberg   London   27                3.0   Project 6   
5   A003     Mr. Tom  Sabestine   Berlin   29                3.0   Project 3   
6   A003     Mr. Tom  Sabestine   Berlin   29                4.0   Project 8   
7   A003     Mr. Tom  Sabestine   Berlin   29                3.0  Project 10   
8   A004   Mrs. Nina      Adgra  Newyork   31                2.0   Project 4   
9   A004   Mrs. Nina      Adgra  Newyork   31                2.0  Project 12   
10  A004   Mrs. Nina      Adgra  Newyork   31                2.0  Project 13   
11  A005    Mrs. Amy      Johny   Madrid

In [13]:
# ===== Task 8 - Promote by 1 if age > 29 using IF condition =====

final['Age'] = pd.to_numeric(final['Age'], errors='coerce')
mask_age = final['Age'] > 29
final.loc[mask_age, 'Designation Level'] = final.loc[mask_age, 'Designation Level'] - 1
final.to_csv('Final_after_promotion.csv', index=False)
print(final)

      ID  First Name  Last Name     City  Age  Designation Level     Project  \
0   A001    Mr. John      Alter    Paris   25                2.0   Project 1   
1   A001    Mr. John      Alter    Paris   25                2.0   Project 9   
2   A001    Mr. John      Alter    Paris   25                3.0  Project 11   
3   A002  Mrs. Alice  Luxumberg   London   27                2.0   Project 2   
4   A002  Mrs. Alice  Luxumberg   London   27                3.0   Project 6   
5   A003     Mr. Tom  Sabestine   Berlin   29                3.0   Project 3   
6   A003     Mr. Tom  Sabestine   Berlin   29                4.0   Project 8   
7   A003     Mr. Tom  Sabestine   Berlin   29                3.0  Project 10   
8   A004   Mrs. Nina      Adgra  Newyork   31                1.0   Project 4   
9   A004   Mrs. Nina      Adgra  Newyork   31                1.0  Project 12   
10  A004   Mrs. Nina      Adgra  Newyork   31                1.0  Project 13   
11  A005    Mrs. Amy      Johny   Madrid

In [14]:
# ===== Task 9 - Total Project Cost per Employee =====

final['Cost'] = pd.to_numeric(final['Cost'], errors='coerce').fillna(0)
total_costs = final.groupby(['ID','First Name'], as_index=False)['Cost'].sum()
total_costs = total_costs.rename(columns={'Cost':'Total cost'})
TotalProjCost = total_costs[['ID','First Name','Total cost']]
TotalProjCost.to_csv('TotalProjCost.csv', index=False)
print(TotalProjCost)


     ID  First Name  Total cost
0  A001    Mr. John   3327000.0
1  A002  Mrs. Alice   2680000.0
2  A003     Mr. Tom   5150000.0
3  A004   Mrs. Nina   9500000.0
4  A005    Mrs. Amy   3690000.0


In [15]:
# ===== Task 10 - Print employee details whose city contains letter 'o' =====

mask_o = final['City'].str.contains('o', case=False, na=False)
employees_with_o = final[mask_o]

print('Task 10 results: Employees whose city contains the letter "o":')
print(employees_with_o.to_string(index=False))

final.to_csv('Final_completed.csv', index=False)


Task 10 results: Employees whose city contains the letter "o":
  ID First Name Last Name    City  Age  Designation Level    Project      Cost   Status    Bonus
A002 Mrs. Alice Luxumberg  London   27                2.0  Project 2 2000000.0  Ongoing      0.0
A002 Mrs. Alice Luxumberg  London   27                3.0  Project 6  680000.0   Failed      0.0
A004  Mrs. Nina     Adgra Newyork   31                1.0  Project 4 5500000.0  Ongoing      0.0
A004  Mrs. Nina     Adgra Newyork   31                1.0 Project 12 1000000.0  Ongoing      0.0
A004  Mrs. Nina     Adgra Newyork   31                1.0 Project 13 3000000.0 Finished 150000.0
